In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm # this is a fancy progress bar!
from time import sleep
import numpy as np
from datetime import datetime
import pandas as pd

In [ ]:
link_first_part = 'https://www.jobs.ch'
link_mid_1_part = '/en/vacancies/?page='
link_mid_2_part = '&term='

In [ ]:
link = 'https://www.jobs.ch/en/vacancies/?page=1&term=Data%20Engineer'

In [ ]:
response = requests.get(link, timeout = 15)
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
response.status_code

In [ ]:
soup.find('div', class_ = 'Box-sc-7ekkso-0 Position-b2pct5-0 Position__Relative-b2pct5-1 VacancySerpItem__ShadowBox-p4qu0m-0 hthPRS')


In [33]:
job_links = []
def get_links(soup, job_links):
    all_links = soup.find_all('div', class_ = 'Box-sc-7ekkso-0 Position-b2pct5-0 Position__Relative-b2pct5-1 VacancySerpItem__ShadowBox-p4qu0m-0 hthPRS')
    for job_add in all_links:
        job_links.append(link_first_part+job_add.find('a', {'class':'x--job-link t--job-link SearchVacancyResultsComponent__StyledVacancySerpItem-n25jij-0 dQDQbr'}).get('href'))



In [30]:
max_pages = soup.find('div', class_ = 'Div-v2w9ke-0 Flex-sc-4aokm-0 eykbax').text.split()[2]

In [36]:
for page in tqdm(range(1, int(max_pages)+1)[:10]):
    url = link_first_part +link_mid_1_part + str(page) + link_mid_2_part + 'Data%20Engineer'
    response = requests.get(url, timeout = 15)
    soup = BeautifulSoup(response.content, 'html.parser')
    get_links(soup, job_links)
    sleep(0.6)
    

In [72]:
job_links

['https://www.jobs.ch/en/vacancies/detail/9309672/?jobposition=1-1&source=vacancy_search_promo',
 'https://www.jobs.ch/en/vacancies/detail/9321833/?jobposition=1-2&source=vacancy_search_promo',
 'https://www.jobs.ch/en/vacancies/detail/9333308/?jobposition=1-3&source=vacancy_search',
 'https://www.jobs.ch/en/vacancies/detail/9317411/?jobposition=1-4&source=vacancy_search',
 'https://www.jobs.ch/en/vacancies/detail/9319482/?jobposition=1-5&source=vacancy_search',
 'https://www.jobs.ch/en/vacancies/detail/9332216/?jobposition=1-6&source=vacancy_search',
 'https://www.jobs.ch/en/vacancies/detail/9326033/?jobposition=1-7&source=vacancy_search',
 'https://www.jobs.ch/en/vacancies/detail/8824923/?jobposition=1-8&source=vacancy_search',
 'https://www.jobs.ch/en/vacancies/detail/9328560/?jobposition=1-9&source=vacancy_search',
 'https://www.jobs.ch/en/vacancies/detail/9191812/?jobposition=1-10&source=vacancy_search',
 'https://www.jobs.ch/en/vacancies/detail/9336861/?jobposition=1-11&source=va

In [73]:
job_link = 'https://www.jobs.ch/en/vacancies/detail/9309672/?jobposition=1-1&source=vacancy_search_promo'

In [74]:
response = requests.get(job_link, timeout = 15)
soup = BeautifulSoup(response.content, 'html.parser')


In [75]:
def get_content(soup):
    try:
        content = soup.find('div', class_ = 'Div-v2w9ke-0 fjQgMg').get_text()
    except:
        content = np.nan
    try:    
        title = soup.find('div', {'class' : 'Div-v2w9ke-0 hPuVjT'}).find('h1').get('title')
    except:
        title = np.nan
    try:
        company = soup.find('div', class_ = 'Div-v2w9ke-0 cvwY').find('a').get('title')
    except:
        company = np.nan
    try:
        published = soup.find('span', class_ = 'Span-bhy2uh-0 Badge-ndaeev-0 krRxxu').get_text()
    except: 
        published = np.nan
    return content, title, company, published

In [76]:
get_content(soup)

('Job descriptionInfoCompanyMöchtest Du Teil der KLARA Erfolgsgeschichte werden und deinen Beitrag zur Digitalisierung der Schweizer KMU Landschaft beisteuern, dann bewirb dich jetzt. Wir suchen in Zug eine/n KLARA Maschine Learning Engineer 80 -100%KLARA Machine Learning Engineer | Zug        \xa0   Standort Zug            \xa0        KLARA MACHINE LEARNING ENGINEER 80 - 100% (w/m)            Möchtest Du Teil der KLARA Erfolgsgeschichte werden und deinen Beitrag zur Digitalisierung der Schweizer KMU Landschaft beisteuern, dann bewirb dich jetzt. Wir suchen in Zug eine/n KLARA Maschine Learning Engineer 80 -100%   Deine Aufgaben   Du spielst eine Schlüsselrolle bei der Entwicklung der Intelligenz in KLARA. Mit Deiner Erfahrung im Bereich des maschinellen Lernens und in der Software Entwicklung realisierst du echten Mehrwert für unsere Nutzer. Bei grossen Datenmengen behältst Du den Durchblick. Die Identifikation geeigneter Algorithmen und Ansätze zur Lösung maschineller Lernprobleme li

In [77]:
cols = ['date', 'company', 'title', 'published', 'content']
df = pd.DataFrame(columns = cols)
df

,date,company,title,published,content


In [78]:
for job in tqdm(job_links[:20]):
    response = requests.get(job, timeout = 15)
    soup = BeautifulSoup(response.content, 'html.parser')
    content, title, company, published = get_content(soup)
    
    df = df.append({
        'date': datetime.now(), 
        'company': company, 
        'title': title, 
        'published': published, 
        'content': content
        
        
    }, ignore_index = True)
    sleep(0.6)

df
    

,date,company,title,published,content
0,2019-11-15 12:38:55.859578,Klara Business AG,KLARA Machine Learning Engineer | Zug,31.10.2019,Job descriptionInfoCompanyMöchtest Du Teil der...
1,2019-11-15 12:38:57.094772,Syngenta,Experienced Packaging Engineer (m/f/n),06.11.2019,Job descriptionInfoCompany
2,2019-11-15 12:38:58.255294,The Stamford Group AG,DevOps Engineer,13.11.2019,Job descriptionInfoCompanyDevOps EngineerJob D...
3,2019-11-15 12:38:59.655556,Experis Schweiz Zürich,Senior Data Engineer,04.11.2019,Job descriptionInfoCompanySenior Data Engineer...
4,2019-11-15 12:39:00.992014,uniqFEED AG,DevOps / SRE (80-100%),05.11.2019,"Job descriptionInfoCompanyuniqFEED, a Spin-off..."
5,2019-11-15 12:39:02.247132,RM IT Professional Resources AG,Zendesk Engineer,12.11.2019,Job descriptionInfoCompanyZendesk Engineer vac...
6,2019-11-15 12:39:03.650979,Raiffeisen Gruppe,Big Data Engineer (w/m),08.11.2019,Job descriptionInfoCompanyWerden Sie Teil eine...
7,2019-11-15 12:39:04.878702,Accenture,Entwicklerin (all genders) DevOps,30.10.2019,Job descriptionInfoCompany
8,2019-11-15 12:39:06.008971,Leica Geosystems AG,Machine Learning Engineer (f/m),09.11.2019,Job descriptionInfoCompanyMachine Learning Eng...
9,2019-11-15 12:39:07.204630,Credit Suisse AG,Data Visualisation Engineer 80 - 100%,04.11.2019,Job descriptionInfoCompanyData Visualisation E...
